Analyzing Media Coverage Of Other Countries in Austria
===


The aim of this project is to find out how reporting in Austria's print media about certain countries has changed over time (sentiment, which topics). In two further steps, I would also like to go down to the level of the individual newspapers and their authors and take an explorative look at whether there are tendencies/bias here. As countries of interest I choose Balkan countries because of their proximity to Austria and their long history of conflict.

    Text corpora: Austrian newspaper articles (or their respective twitter posts)
    Language: German
    Time: 2000–2022
    Method: Adding country labels to the articles, Sentiment Scores (, extracting underlying topics of the articles, e.g. with fuzzy topic modeling) 
    
According to https://de.wikipedia.org/wiki/Liste_%C3%B6sterreichischer_Zeitungen_und_Zeitschriften derstandard.at and krone.at reach the most people online, they also cover Austrian society quite well. So, initially, I will focus on these two and might add some more newspapers at a later stage.

* Focus on 2 countries: Serbia and Croatia (like Austria now a EU country)
* I will label an article with "Serbia" if the ratio of amount of words that relate to "Serbia"  compared to words that relate to "Croatia" is greater than 4
* I will also try to find out if there are certain authors who are responsible for a certain tendency.
* Another analysis could be build upon the ratio of "Serbia" and "Albania" or "Kosovo" 

In [1]:
import jupyter_black

jupyter_black.load()

In [27]:
import os
import json
import subprocess
import pandas as pd


data_dir = "data/twitter"

if os.path.exists(data_dir) == False:
    os.makedirs(data_dir)


def dl_user(user, max_results=None, local=False):
    """
    Function to download tweets by username.
    Set local to True, if tweets have already
    been downloaded and are available in data_dir.

    Returns a DataFrame.

    """

    if local == False:
        with open(data_dir + f"/user-{user}.json", "w+") as fo:
            if max_results == None:
                cmd_list = ["snscrape", "--jsonl", "twitter-user", user]
            else:
                cmd_list = [
                    "snscrape",
                    "--jsonl",
                    "-n " + str(max_results),
                    "twitter-user",
                    user,
                ]
            p = subprocess.Popen(cmd_list, stdout=fo)
            p.wait()

    with open(data_dir + f"/user-{user}.json", "r") as fo:
        tweets = fo.readlines()

    tweets = [json.loads(tweets[i]) for i in range(0, len(tweets))]
    print("loaded", len(tweets), "tweets\n")

    df_tweets = pd.DataFrame(tweets)
    df_tweets["date"] = pd.to_datetime(df_tweets["date"])

    return df_tweets

# 1. Scraping newspaper articles

## 1.1. derStandard.at


In [28]:
df_tweets = dl_user(
    "derstandardat", max_results=None, local=True
)  # max_results=None for all (default)

# print the first 5
df_tweets[["date", "rawContent", "hashtags"]].head()

loaded 272221 tweets



,date,rawContent,hashtags
0,2023-01-24 20:00:04+00:00,Kantersieg für Leipzig gegen Schalke: https:/...,None
1,2023-01-24 19:31:27+00:00,Mordverdacht in Mürzzuschlag: Frau tödlich ver...,None
2,2023-01-24 18:57:26+00:00,Welche Laufgruppen gibt es in Wien?: https://...,None
3,2023-01-24 18:57:25+00:00,Gleichberechtigung beim Dating: Bumble-Gründer...,None
4,2023-01-24 18:43:22+00:00,Aigner und Edlinger holen Gold bei Para-WM: h...,None


In [26]:
df_tewee